In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

from tqdm.auto import tqdm
import requests
import zipfile
import pathlib
from pathlib import Path
import shutil
import os
import pandas as pd
import numpy as np
import random
from PIL import Image
from sklearn.model_selection import StratifiedGroupKFold
from typing import Tuple, Dict, List

import CarImageClass

device = "cuda" if torch.cuda.is_available() else "cpu"

train_path = r'D:/Udacity_car_data/data/train'
test_path = r'D:/Udacity_car_data/data/test'

train_path_simple = r'D:/Udacity_car_data/data/train_simple'
test_path_simple = r'D:/Udacity_car_data/data/test_simple'

In [26]:
# Create transform (no data augmentation)
data_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [27]:
# 1. Load and transform data
train_data_simple = CarImageClass.ImageClassSimple(targ_dir=train_path_simple, transform=data_transform)
test_data_simple = CarImageClass.ImageClassSimple(targ_dir=test_path_simple, transform=data_transform)

# 2. Turn data into DataLoaders
# Setup batch size and number of workers 
BATCH_SIZE = 32
#NUM_WORKERS = os.cpu_count()
#print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     #num_workers=NUM_WORKERS,
                                     )

test_dataloader_simple = DataLoader(test_data_simple, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    #num_workers=NUM_WORKERS,
                                    )

train_dataloader_simple, test_dataloader_simple

(<torch.utils.data.dataloader.DataLoader at 0x197a76c05c0>,
 <torch.utils.data.dataloader.DataLoader at 0x197ae065bb0>)